In [1]:
import win32com.client
import json
import re

# Outlook関係のオブジェクト初期化
app = win32com.client.Dispatch("Outlook.Application")
root = app.Session.DefaultStore.GetRootFolder()
ns = app.GetNamespace("MAPI")
inbox = ns.GetDefaultFolder(6)
messages = inbox.Items
print("inbox:", inbox.folderpath)

inbox: \\Jiro.Kogi@jp.nttdata.com\受信トレイ


In [2]:
# Outlookのフォルダ検索
def findfolder(root, name):
    for folder in root.Folders:
        if name in folder.name:
            print(folder.folderpath)
            return folder
        ret = findfolder(folder, name)
        if ret is not None:
            return ret
    return None


# 条件適合判定
def isit(message, subjects, addresses=[]):
    for subject in subjects:
        # if subject in message.subject:
        #    return True
        if re.search(subject, message.subject) != None:
            return True
    for address in addresses:
        if address in message.sendername:
            return True
        if address in message.senderemailaddress:
            return True
        for recip in message.recipients:
            if address in recip.name or address in recip.address:
                return True
    return False


# アーカイブ先フォルダ検索
def whichFolder(message, dic):
    if message.messageClass == "IPM.Note":
        for key in dic:
            if isit(message, dic[key]["subject"], dic[key]["address"]):
                return key
    return None


# JSONファイルから移動条件をロードする
def load_json(filename="mail.json"):
    with open("mail.json", "r", encoding="utf-8") as f:
        dic = json.load(f)
        # print(type(dic))
        folders = {}
        for k in dic:
            # print(k,dic[k]["unread"])
            if not k.startswith("$"):
                folders[k] = findfolder(root, dic[k]["folder"])
    return dic, folders

In [3]:
# メールのアーカイブ処理メイン部
def move_mail(
    dic, folders, target_folder=inbox, view_none=True, view_move=True, view_delete=True
):
    i = 1
    counter_move = 0
    counter_remain = 0
    counter_delete = 0
    counter_unread = 0
    counter_not_msg = 0
    list_move = list()
    for message in target_folder.Items:
        key = whichFolder(message, dic)
        # print(key)
        if key == "del":
            counter_delete += 1
            if view_none:
                print(counter_delete, "delete", message.subject)
            list_move.append((message, None))
        elif (
            key is None
            or folders[key] is None
            or folders[key].folderpath == target_folder.folderpath
        ):
            if message.messageClass != "IPM.Note":
                counter_not_msg += 1
                counter_remain += 1
                if view_none:
                    print(counter_remain, "apnt", message.subject)
            elif message.unread:
                counter_remain += 1
                counter_unread += 1
                if view_none:
                    print(counter_remain, "unread", message.subject)
            else:
                counter_remain += 1
                if view_none:
                    print(counter_remain, "none", message.subject)
        elif dic[key]["unread"] or not message.unread:
            counter_move += 1
            if view_move:
                print(folders[key].name, message.subject)
            list_move.append((message, folders[key]))
        else:
            counter_remain += 1
            counter_unread += 1
            if view_none:
                print("unread", message.subject)
        i += 1
    for item in list_move:
        message = item[0]
        dest = item[1]
        if dest is None:
            if view_delete:
                print("delete", message.subject)
            message.delete()
        else:
            print(dest.name, message.subject)
            message.unread = False
            message.move(dest)
    print("moved:", counter_move, "delete:", counter_delete, "remain:", counter_remain)
    print("unread",counter_unread, "appoint:", counter_not_msg)


# アーカイブ処理を全アーカイブ対象フォルダに対して実行
def do_all_folder(dic, folders):
    for k in dic:
        print(k)
        if k != "del":
            move_mail(dic, folders, target_folder=folders[k], view_none=False)
    print("do all done.")

In [4]:
# JSONファイル（辞書ファイル）読み込み
dic, folders = load_json()

\\Jiro.Kogi@jp.nttdata.com\#90 Archive
\\Jiro.Kogi@jp.nttdata.com\#33 UX
\\Jiro.Kogi@jp.nttdata.com\#05 PMO\10 故障当番
\\Jiro.Kogi@jp.nttdata.com\#31 故障速報
\\Jiro.Kogi@jp.nttdata.com\#13 関西\08 関西電子申請
\\Jiro.Kogi@jp.nttdata.com\#13 関西\07 関西エアポート
\\Jiro.Kogi@jp.nttdata.com\#13 関西\01 竹中
\\Jiro.Kogi@jp.nttdata.com\#11 九州\01 サニクリーン
\\Jiro.Kogi@jp.nttdata.com\#11 九州\02 LiveCampus
\\Jiro.Kogi@jp.nttdata.com\#11 九州\03 鹿児島県庶務事務
\\Jiro.Kogi@jp.nttdata.com\#11 九州
\\Jiro.Kogi@jp.nttdata.com\#19 JIP
\\Jiro.Kogi@jp.nttdata.com\#20 NDI
\\Jiro.Kogi@jp.nttdata.com\#12 中国(国内)
\\Jiro.Kogi@jp.nttdata.com\#13 関西\02 グリコ
\\Jiro.Kogi@jp.nttdata.com\#13 関西\03 大阪府帳票
\\Jiro.Kogi@jp.nttdata.com\#13 関西\04 大阪市国介
\\Jiro.Kogi@jp.nttdata.com\#13 関西\06 中立電機
\\Jiro.Kogi@jp.nttdata.com\#10 四国\03 玉井歯科商店
\\Jiro.Kogi@jp.nttdata.com\#10 四国\04 愛媛県財務
\\Jiro.Kogi@jp.nttdata.com\#10 四国\01 愛媛庶務事務
\\Jiro.Kogi@jp.nttdata.com\#10 四国\02 兼松エンジニアリング
\\Jiro.Kogi@jp.nttdata.com\#10 四国
\\Jiro.Kogi@jp.nttdata.com\#16 信越\02 i-line
\\Jiro.Kogi@j

In [5]:
# ターゲットフォルダを変数で制御する。
# 空文字の場合は受信トレイ、"all"で全フォルダ
target_name = ""

if target_name == "":
    # 受信トレイ(inbox)に対して処理を行う場合
    move_mail(dic, folders)
elif target_name == "all":
    # 全てのフォルダに対して再処理を行う場合
    do_all_folder(dic,folders)
else:
    # 特定のフォルダに対して処理を行う場合
    tf = findfolder(root, taget_name)
    move_mail(dic, folders, target_folder=tf)

1 none [p_pmo_rc:04675] FW: 重要案件審議会予定の共有
2 none ８月ー９月の重案審日程（月別時間割）につきまして
3 none [xgpsdpps_pmo:00024] 【Zoom有償ライセンス利用の方必読】Zoomのご予約方法の変更について
4 none [xgpsdpps_pmo:00029] セルフ・イノベーションタイムの就業管理システムへの投入をお忘れなく！
5 none  【周知　就業管理】月締めについて（2021年8月稼働）
6 none 【依頼】NTT健保互選代議員議員選挙　投票用紙受領書に関する取扱について
7 none ＊○【投票開始・要対応】2021年度互選代議員・議員選挙について
8 none [p_pmo_rc:05978] FW: 【分野セキュリティMagazine 30】 社内セキュリティ規程類が改正されました
moved: 0 delete: 0 remain: 8
unread 0 appoint: 0
